Import the libraries needed

In [1]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import psutil


import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


C:\Users\m033\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


### Database connections and SQL to retrieve the various data needed.

In [2]:
SQL_server = 'WINPRDAI1572\CLESQLDW'
Reporting_database = 'CLE_Reporting'
Reference_Data_database = 'CLE_Reference_Data'

pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-07-19 19:09:02.496571
svmem(total=16756752384, available=3128823808, percent=81.3, used=13627928576, free=3128823808)


# 1. Get Broker and Commissions features.

  -  Identified in Lit Review.# 1. Read back the population with switches data.

In [3]:
conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+SQL_server+';'
         'DATABASE='+Reporting_database+';'
         'Trusted_Connection=yes;')

broker_query = f"""SELECT pol.Policy_Number AS ContractNumber
	  ,pol.Servicing_Broker_Id AS ServicingBroker
      ,pol.Initial_Broker_Id   AS InitialBroker
      ,pb.Broker_Category_En AS BrokerCategory
	  ,AVG(cr.Initial_Rate) AS InitialRate
	  ,AVG(cr.renewal_Rate) AS RenewalRate
FROM cle_reporting.dbo.CLT_Policy pol
INNER JOIN cle_reporting.dbo.CLT_Product prd ON prd.Product_Id = pol.Product_Id
INNER JOIN  CLE_Reporting.dbo.CLT_Policy_Broker pb ON pb.Statement_Broker_Id = pol.Servicing_Broker_Id
INNER JOIN [CLE_Reporting].[dbo].[CLT_Commission_Rate] cr ON cr.Policy_Broker_Id = pb.Policy_Broker_Type_Id --AND cr.Policy_Broker_Id = pb.Policy_Broker_Id
WHERE prd.Product_Id = pol.Product_Id
AND   prd.Product_Code IN ('FKP_2006V1','FKP_2006V2','FKP_2008V1','FKP_2010V1','FKP_2012V1','FKP_2017V1',
                                            'GNPP_2016V1','GNPP_2017V1','GNPP_2022V1','GNPP_2023V1',
                                            'GNRP_2008V1','GNRP_2010V1','GNRP_2012V1','GNRP_2017V1','GNRP_2022V1',
                                            'XRPP_2005V1','XRPP_2008V1','XRPP_2010V1','XRPP_2012V1','XRPP_2015V1')
AND SUBSTRING(prd.Product_Category_Code,1,3) = SUBSTRING(cr.Commissions_Product_Group,1,3)
GROUP BY pol.Policy_Number,pol.Servicing_Broker_Id,pol.Initial_Broker_Id,pb.Broker_Category_En;"""

df_broker = pd.read_sql(broker_query, conn)

# Types and row count
# print stats.
print(len(df_broker.index))
now = datetime.now()
print(now)
print('==========================')


print(df_broker.dtypes)
df_broker.head(5)

69459
2024-07-19 19:09:06.634077
ContractNumber      object
ServicingBroker      int64
InitialBroker        int64
BrokerCategory      object
InitialRate        float64
RenewalRate        float64
dtype: object


,ContractNumber,ServicingBroker,InitialBroker,BrokerCategory,InitialRate,RenewalRate
0,8153732A,1310,1310,B,4.35,1.07
1,6625275M,13860,13860,B,4.30,0.65
2,6751057K,3748,3748,B,4.38,0.96
3,8050943A,5367,5367,D,4.38,0.96
4,8020403N,2802,2802,D,4.38,0.96


## Drop obsolete or repeated features.

## Check for nulls

In [4]:
df_broker.dtypes

ContractNumber      object
ServicingBroker      int64
InitialBroker        int64
BrokerCategory      object
InitialRate        float64
RenewalRate        float64
dtype: object

In [5]:
df_broker.isna().any()

ContractNumber     False
ServicingBroker    False
InitialBroker      False
BrokerCategory     False
InitialRate        False
RenewalRate        False
dtype: bool

## No NULLS so check the distributions of values for each column.

In [6]:
print('ServicingBroker : ',df_broker['ServicingBroker'].value_counts())

ServicingBroker :  5683     9134
2571     6389
3353     3976
2650     2712
16102    1792
16535    1615
14310    1494
3754     1470
16586    1262
11483     911
9         880
8721      678
173       641
3896      596
5494      578
5431      535
12517     476
14281     471
5367      464
12906     434
683       431
500       412
17444     399
12050     353
19857     351
5125      348
334       342
20745     339
1138      307
6620      279
13603     278
3748      272
3132      248
8931      242
16803     242
19313     224
3900      219
4268      213
9604      211
3950      202
9207      193
21236     184
13481     182
282       179
2422      165
309       158
12230     157
3647      157
3323      154
3409      147
935       146
2860      146
337       141
9689      140
9775      136
14122     131
9732      128
20691     128
5495      128
14458     128
255       128
4124      126
8848      119
18466     119
1207      116
7164      114
9661      112
9616      112
4392      111
5722      108
1

In [7]:
print('InitialBroker   : ',df_broker['InitialBroker'].value_counts())

InitialBroker   :  5683     9132
2571     6390
3353     3976
2650     2712
16102    1792
16535    1616
14310    1494
3754     1470
16586    1262
11483     911
9         880
8721      678
173       641
3896      596
5494      578
5431      535
12517     476
14281     471
5367      464
12906     434
683       431
500       412
17444     399
12050     353
19857     351
5125      348
334       342
20745     338
1138      307
6620      279
13603     278
3748      272
3132      248
8931      242
16803     242
19313     223
3900      219
4268      213
9604      211
3950      202
9207      193
21236     184
13481     182
282       179
2422      165
309       158
12230     157
3647      157
3323      154
3409      147
2860      146
935       146
337       141
9689      140
9775      136
14122     131
20691     128
14458     128
5495      128
9732      128
255       128
4124      126
18466     119
8848      119
1207      116
7164      114
9616      112
9661      112
4392      111
5722      108
1

In [8]:
print('InitialRate     : ',df_broker['InitialRate'].value_counts())

InitialRate     :  4.3800    29405
4.5000    21898
4.2500     6079
4.3500     4423
4.4500     2329
4.2333     1275
4.3200      437
4.2250      387
4.4000      387
4.0666      355
4.0000      317
4.4250      245
4.1666      244
4.2573      184
4.4666      179
4.4100      155
4.3600       80
4.2235       73
4.2111       73
4.4750       65
4.2000       58
3.8090       55
4.1333       53
4.2463       50
4.2040       50
4.3650       42
4.1800       41
4.3000       40
4.4978       38
3.9687       34
4.1269       31
4.3920       30
4.4333       21
4.2800       20
4.4800       20
4.3350       19
4.4400       19
4.3950       15
4.4840       15
4.4993       13
4.4600       13
4.3680       12
4.3333       12
4.1250       11
3.9875       11
4.3400       10
4.1900        8
4.3796        8
4.2375        7
4.3529        6
4.1214        6
4.1555        6
4.1500        6
4.2498        6
3.3333        6
4.2146        6
4.4200        5
4.3300        5
4.3761        5
4.4416        5
4.3166        4
4.270

In [9]:
print('RenewalRate     : ',df_broker['RenewalRate'].value_counts())

RenewalRate     :  0.9600    29345
0.8250    20915
0.6250     6031
1.0700     4334
0.9625     2254
0.7375     1274
0.2000     1093
1.1800      437
0.9166      399
0.7937      385
1.1000      372
0.9666      355
0.9000      322
0.7166      233
1.0163      184
0.8937      146
1.2300      127
1.0333       80
0.7750       79
0.8055       73
0.8727       55
0.9416       53
0.6462       50
0.7600       50
1.0312       47
1.1120       45
1.0083       43
0.8150       42
1.0150       42
0.8279       38
0.9687       34
0.9653       31
1.0680       30
1.0400       28
1.2800       24
1.1200       24
0.9350       24
1.1400       23
0.6725       22
1.0950       21
1.1250       19
0.7607       19
0.6800       16
0.8468       15
0.8800       15
0.8266       13
1.0040       12
0.6916       11
0.7850        9
0.7625        8
0.7325        8
0.9613        8
0.7093        7
1.0666        7
0.1875        7
0.8041        6
0.9785        6
0.9764        6
0.6273        6
1.0682        6
0.8000        6
0.962

## All features well distributed so write to csv.

In [10]:
# Write out to a comma separated values file.

# giving directory name
filename = input_path + '\Broker_and_Commission_data.csv'
df_broker.to_csv(filename, encoding='utf-8', index=False)        
df_broker.head(10)

,ContractNumber,ServicingBroker,InitialBroker,BrokerCategory,InitialRate,RenewalRate
0,8153732A,1310,1310,B,4.35,1.070
1,6625275M,13860,13860,B,4.30,0.650
2,6751057K,3748,3748,B,4.38,0.960
3,8050943A,5367,5367,D,4.38,0.960
4,8020403N,2802,2802,D,4.38,0.960
5,6561873Y,5683,5683,D,4.50,0.825
6,6794647A,20691,20691,D,4.50,0.825
7,8203782R,9655,9655,D,4.38,0.960
8,8172424J,237,237,D,4.38,0.960
9,8050230T,3353,3353,D,4.35,1.070


In [11]:
%who_ls DataFrame 

['df_broker']

In [12]:
del df_broker

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')



2024-07-19 19:09:07.695844
svmem(total=16756752384, available=3064979456, percent=81.7, used=13691772928, free=3064979456)
